In [87]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, date, time
import numpy as np

my_openweather_api_key = '888888888888888888888888'
my_RapidAPI_key = '8888888888888888888888'

In [24]:
# get timestamp in correct format for tomorrow 8 AM until 8 PM
def get_tomorrow():
    date_tomorrow = date.today() + timedelta(days=1)
    time_start = time(hour=0, minute=0, second=0)
    time_middle = time(hour=12, minute=0, second=0)
    time_end = time(hour=23, minute=59, second=59)

    datetime_start = datetime.combine(date_tomorrow, time_start).strftime('%Y-%m-%dT%H:%M')
    datetime_middle = datetime.combine(date_tomorrow, time_middle).strftime('%Y-%m-%dT%H:%M')
    datetime_end = datetime.combine(date_tomorrow, time_end).strftime('%Y-%m-%dT%H:%M')
    return datetime_start,datetime_middle,datetime_end

In [112]:
def weather_forcast(list_of_cities):
   
    cities_list  = []
    weather_list = []
    
    for city in list_of_cities :
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={my_openweather_api_key}&units=metric")
        
        if city_r.status_code != 200:
            return np.nan
        
        
        city_j = city_r.json()
        
        cities_list.append(city_j['city'])
        

        for detail in city_j['list']:
            weather_details_dic = {}
            weather_details_dic["city_id"] = city_j['city']['id']
            weather_details_dic["city_name"] = city_j['city']['name']
            weather_details_dic["temp"] = detail['main']['temp']
            weather_details_dic["temp_min"] = detail['main']['temp_min']
            weather_details_dic["temp_max"] = detail['main']['temp_max']
            weather_details_dic["feels_like"] = detail['main']['feels_like']
            weather_details_dic["pressure"] = detail['main']['pressure']
            weather_details_dic["humidity"] = detail['main']['humidity']
            weather_details_dic["weather"] = detail['weather'][0]['main']
            weather_details_dic["weather_description"] = detail['weather'][0]['description']
            weather_details_dic["clouds"] = detail['clouds']['all']
            weather_details_dic["wind_speed"] = detail['wind']['speed']
            weather_details_dic["date_time_3h"] = detail['dt_txt']
            weather_list.append(weather_details_dic)
    
    
    df_weather = pd.DataFrame(weather_list)
    dt_start,dt_middle,dt_end = get_tomorrow()
    df_weather['date_time_3h'] = pd.to_datetime(df_weather['date_time_3h'])
    df_weather = df_weather.loc[lambda df_ : (df_['date_time_3h'] >= dt_start) &  (df_['date_time_3h'] <= dt_end)]
    
    
    df_cities = pd.json_normalize(cities_list)
    df_cities.rename(columns={'id':'city_id'}, inplace=True)
    
    return df_weather,df_cities

In [111]:
def get_citys_airports(city_name, country_iso2, city_id):
    
	url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

	querystring = {"q":city_name,"limit":"10"}

	headers = {
		"X-RapidAPI-Key": my_RapidAPI_key,
		"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
	}

	airports_r = requests.request("GET", url, headers=headers, params=querystring)
 
	if airports_r.status_code != 200:
		return np.nan 
    
	airports_j = airports_r.json()

	df_airports =pd.json_normalize(airports_j['items'])
	df_airports = df_airports.loc[lambda df_ : df_['countryCode'] == country_iso2]
	df_airports.rename(columns={'name':'airport_name', 'municipalityName': 'city_name'}, inplace=True)
	df_airports['city_id'] = city_id
	
	return df_airports

In [205]:
def get_arrival_flights(icao, city_id):

    # make api call to receive flight information
    result_df = pd.DataFrame(columns=['flight_NO', 'flight_status', 'origin_ap_name', 'origin_ap_icao', 'origin_ap_iata', 'scheduledTimeLocal',
                                    'scheduledTimeUtc',
                                    'airline_name','aircraf_model', 'city_id'])
    
    dt_start,dt_middle,dt_end = get_tomorrow()
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{dt_start}/{dt_middle}"

    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

    headers = {
        "X-RapidAPI-Key": my_RapidAPI_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    arrivals_r = requests.request("GET", url, headers=headers, params=querystring)


    if arrivals_r.status_code != 200:
        return result_df
    
    arrivals_j = arrivals_r.json()
    
    if len(arrivals_j['arrivals'])==0:
        return result_df
    
    arrivals_df = pd.json_normalize(arrivals_j['arrivals']).sort_values(by='movement.scheduledTimeLocal')

    # clean df
    # arrivals_df = arrivals_df.loc[lambda df_ : df_['isCargo']== False]
    arrivals_df = arrivals_df[['number', 'status', 
                    'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                    'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc',
                    'airline.name', 'aircraft.model']]
    rename_dict= {'number':'flight_NO', 'status':'flight_status', 
                    'movement.airport.name' : 'origin_ap_name','movement.airport.icao' : 'origin_ap_icao', 'movement.airport.iata':'origin_ap_iata', 
                    'movement.scheduledTimeLocal' : 'scheduledTimeLocal', 'movement.scheduledTimeUtc' : 'scheduledTimeUtc', 
                    'airline.name':'airline_name', 'aircraft.model' :'aircraf_model'}
    arrivals_df.rename(columns=rename_dict , inplace=True)
    arrivals_df['city_id']=city_id

    return arrivals_df


In [114]:
#PART1
weather_df, cities_df = weather_forcast(['LONDON', 'BERLIN'])

In [117]:
cities_df

,city_id,name,country,population,timezone,sunrise,sunset,coord.lat,coord.lon
0,2643743,London,GB,1000000,3600,1659414327,1659469679,51.5085,-0.1257
1,2950159,Berlin,DE,1000000,7200,1659410837,1659466672,52.5244,13.4105


In [204]:
#PART2

airports_df = pd.DataFrame(columns=['icao','iata','airport_name','shortName','city_name','countryCode','location.lat','location.lon', 'city_id'])

for index,city in cities_df.iterrows():
    city_airports = get_citys_airports(city['name'] ,city['country'],city['city_id'])
    airports_df = pd.concat([airports_df,city_airports])
airports_df.reset_index(drop=True, inplace=True)
airports_df

,icao,iata,airport_name,shortName,city_name,countryCode,location.lat,location.lon,city_id
0,EGSS,STN,"London, London Stansted",Stansted,London,GB,51.88500,0.234999,2643743
1,EGLL,LHR,"London, London Heathrow",Heathrow,London,GB,51.47060,-0.461941,2643743
2,EGLC,LCY,"London, London City",City,London,GB,51.50530,0.055277,2643743
3,EGKK,LGW,"London, London Gatwick",Gatwick,London,GB,51.14810,-0.190277,2643743
4,EGGW,LTN,"London, London Luton",Luton,London,GB,51.87470,-0.368333,2643743
5,EGKB,BQH,"London, London Biggin Hill",Biggin Hill,London,GB,51.33080,0.032499,2643743
6,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889,2950159
7,EDDT,TXL,"Berlin, Berlin-Tegel",-Tegel,Berlin,DE,52.55970,13.287699,2950159


In [199]:
#PART3
arrival_flights_df = pd.DataFrame(columns=['flight_NO', 'flight_status', 'origin_ap_name', 'origin_ap_icao', 'origin_ap_iata','scheduledTimeLocal',
                                    'scheduledTimeUtc','airline_name','aircraf_model', 'city_id'])
for index,airport in airports_df.iterrows():
    arrival_flights = get_arrival_flights(airport['icao'] , airport['city_id'])
    arrival_flights_df = pd.concat([arrival_flights_df,arrival_flights])
arrival_flights_df.reset_index(drop=True, inplace=True)
arrival_flights_df

,flight_NO,flight_status,origin_ap_name,origin_ap_icao,origin_ap_iata,scheduledTimeLocal,scheduledTimeUtc,airline_name,aircraf_model,city_id
0,LS 1482B,Expected,Kos,NaN,NaN,2022-08-03 00:00+01:00,2022-08-02 23:00Z,Jet2,NaN,2643743
67,FR 8348,Expected,Porto,LPPR,OPO,2022-08-03 00:00+01:00,2022-08-02 23:00Z,Ryanair,Boeing 737-800,2643743
39,FR 8165,Expected,Málaga,LEMG,AGP,2022-08-03 00:00+01:00,2022-08-02 23:00Z,Ryanair,Boeing 737-800 (winglets),2643743
32,RK 3034,Expected,Agadir,GMAD,AGA,2022-08-03 00:05+01:00,2022-08-02 23:05Z,Ryanair UK,Boeing 737-800,2643743
49,FR 8583,Expected,Thessaloniki,LGTS,SKG,2022-08-03 00:10+01:00,2022-08-02 23:10Z,Ryanair,Boeing 737-800,2643743
...,...,...,...,...,...,...,...,...,...,...
34,AF 1734,Unknown,Paris,LFPG,CDG,2022-08-03 11:30+02:00,2022-08-03 09:30Z,Air France,Airbus A220-300,2950159
20,KL 1823,Unknown,Amsterdam,EHAM,AMS,2022-08-03 11:30+02:00,2022-08-03 09:30Z,KLM,Boeing 737-900,2950159
50,XQ 160,Unknown,Antalya,LTAI,AYT,2022-08-03 11:35+02:00,2022-08-03 09:35Z,SunExpress,Boeing 737-800,2950159
36,U2 4604,Unknown,Thessaloniki,LGTS,SKG,2022-08-03 11:40+02:00,2022-08-03 09:40Z,easyJet,Airbus A320,2950159


In [206]:
from IPython.display import JSON
JSON(response.json())

<IPython.core.display.JSON object>